In [890]:
import regex as re
import pandas as pd

In [891]:
try:
    with open("uu13-1999.txt", encoding="utf8") as file:
        file = file.read()
except:
    with open("uu13-1999.txt") as file:
        file = file.read()

## Remove Header

In [892]:
file_no_header = re.sub("\\nPRESIDEN\\nREPUBLIK INDONESIA\\n- \p{N}+ -", " ", file, flags=re.DOTALL)
file_no_header = re.sub("(\\n[^\\n]*? \…[^\\n]*)(?=\\n)", " ", file_no_header, flags=re.DOTALL)

In [893]:
file_no_header

'PRESIDEN\nREPUBLIK INDONESIA\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 13 TAHUN 1999\nTENTANG\nPEMBENTUKAN KABUPATEN DAERAH TINGKAT II LUWU UTARA\nDENGAN RAHMAT TUHAN YANG MAHA ESA\nPRESIDEN REPUBLIK INDONESIA,\nMenimbang : a. bahwa berhubung dengan perkembangan dan kemajuan Propinsi\nDaerah Tingkat I SUlawesi Selatan pada umumnya dan Kabupaten\nDaerah Tingkat II Luwu pada khususnya, dan adanya tuntutan\naspirasi masyarakat, dipandang perlu untuk meningkatkan\npenyelenggaraan pemerintahan, pelaksanaan pembangunan, dan\npembinaan kemasyarakatan guna menjamin perkembangan dan\nkemajuan dimaksud pada masa mendatang;\nb. bahwa sehubungan dengan hal tersebut di atas dan memperhatikan\nperkembangan jumlah penduduk, luas wilayah potensi ekonomi dan\nmeningkatnya beban tugas serta volume kerja di bidang\npemerintahan, pembangunan, dan kemasyarakatan di Kabupaten\nDaerah Tingkat II Luwu, dipandang perlu membentuk Kabupaten\nDaerah Tingkat II Luwu Utara sebagai pemekaran dari Kabupaten\nDaerah T

## Pisah Penjelasan

In [894]:
penjelasan = re.search("Agar setiap orang mengetahuinya.*", file_no_header, re.DOTALL).group()
badan = file_no_header.replace(penjelasan, "")

In [895]:
penjelasan = re.sub("TAMBAHAN LEMBARAN NEGARA.*", "", penjelasan, flags=re.DOTALL)

# Badan

## Ambil BAB

In [896]:
def ambil_bab(text):
    pattern = "(?<=\\n)(BAB\s*[IVXLCDM]+.*?)(?=\\nBAB|$)"
    bab = re.findall(pattern, text, re.DOTALL)
    if bab:
        ## ambil nomor dan nama bab
        pattern_no = "(?<=BAB)(?:\s*)([IVXLCDM]+)"
        pattern_nama = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n)(.*?)(?=\\n\p{Lu}\p{Ll})"
        pattern_isi = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n.*?)(?:\\n)(\p{Lu}\p{Ll}.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in bab]
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) if re.search(pattern_nama, i, re.DOTALL) else "ERROR" for i in bab]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in bab]
        return no, nama, isi
    else:
        no = [""]
        nama = [""]
        isi = [text]
        return no, nama, isi

Penjelasan
1. (?<=\\n)(BAB [IVXLCDM]+.*?)(?=\\nBAB|$)<br>
Digunakan untuk mengambil semua bab<br>
Kondisi:<br>
    a. Text dimulai dengan \n tanpa mengambil \n<br>
    b. Setelah \n dimulai dengan BAB(spasi)[Bilangan Romawi 1 digit dapat diulang beberapa kali] contoh (BAB IIV, BAB VVIX) akan diambil<br>
       BILANGAN ROMAWI YANG TIDAK VALID DAPAT DIAMBIL<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. pengambilan karakter akan berhenti ketika bertemu \nBAB<br>

2. (?<=BAB)(?:\s*)([IVXLCDM]+)<br>
Digunakan untuk mengambil no bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas [Bilangan romawi 1 digit dapat diulang beberapa kali] akan diambil<br>
       BILANGAN ROMAWI YANG TIDAK VALID DAPAT DIAMBIL<br>

3. (?<=BAB)(?:\s*[IVXLCDM]+\s*\\n)([\p{Lu}\s\\n]*)(?=\\n)<br>
Digunakan untuk mengambil nama bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali)[Bilangan Romawi 1 digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n<br>
    b. Mengambil semua karakter berupa huruf kapital atau spasi atau \n<br>
    c. Pengambilan karakter akan berhenti ketika bertemu \n<br>

4. (?<=BAB)(?:\s*[IVXLCDM]+\s*\\n[\p{Lu}\s\\n]*)(?:\\n)(.*)<br>
Digunakan untuk mengambil isi bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali)[Bilangan Romawi 1 digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n<br>
    b. Setelah kondisi di atas teks dimulai dengan huruf kapital atau spasi atau \n hingga berhenti pada \n tanpa diambil (nama bab)<br>
    c. Semua karakter sesudahnya diambil<br>

In [897]:
no_bab, nama_bab, isi_bab = ambil_bab(badan)
df_bab = {"no_bab": no_bab, "nama_bab": nama_bab, "isi_bab": isi_bab}

In [898]:
badan

'PRESIDEN\nREPUBLIK INDONESIA\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 13 TAHUN 1999\nTENTANG\nPEMBENTUKAN KABUPATEN DAERAH TINGKAT II LUWU UTARA\nDENGAN RAHMAT TUHAN YANG MAHA ESA\nPRESIDEN REPUBLIK INDONESIA,\nMenimbang : a. bahwa berhubung dengan perkembangan dan kemajuan Propinsi\nDaerah Tingkat I SUlawesi Selatan pada umumnya dan Kabupaten\nDaerah Tingkat II Luwu pada khususnya, dan adanya tuntutan\naspirasi masyarakat, dipandang perlu untuk meningkatkan\npenyelenggaraan pemerintahan, pelaksanaan pembangunan, dan\npembinaan kemasyarakatan guna menjamin perkembangan dan\nkemajuan dimaksud pada masa mendatang;\nb. bahwa sehubungan dengan hal tersebut di atas dan memperhatikan\nperkembangan jumlah penduduk, luas wilayah potensi ekonomi dan\nmeningkatnya beban tugas serta volume kerja di bidang\npemerintahan, pembangunan, dan kemasyarakatan di Kabupaten\nDaerah Tingkat II Luwu, dipandang perlu membentuk Kabupaten\nDaerah Tingkat II Luwu Utara sebagai pemekaran dari Kabupaten\nDaerah T

In [899]:
df_bab = pd.DataFrame(df_bab)
df_bab

,no_bab,nama_bab,isi_bab
0,I,KETENTUAN UMUM,Pasal 1\nDalam Undang-undang ini yang dimaksud...
1,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",Pasal 2\nDengan Undang-undang ini dibentuk Kab...
2,III,PEMERINTAH DAERAH DAN PERANGKAT WILAYAH/DAERAH,Pasal 8 \nUntuk memimpin jalannya pemerintahan...
3,IV,URUSAN RUMAH TANGGA DAERAH,Pasal 11\n(1) Pada saat terbentuknya Kabupaten...
4,V,KETENTUAN PERALIHAN,Pasal 12\nPada saat terbentuknya Kabupaten Dae...
5,VI,KETENTUAN PENUTUP,Pasal 17\nPada saat berlakunya Undang-undang i...


## Ambil Bagian

In [900]:
def ambil_bagian(text):
    pattern = "(?<=\\n|^)(Bagian\s*\p{Lu}.*?)(?=\\nBagian\s*\p{Lu}|$)"
    bagian = re.findall(pattern, text, re.DOTALL)
    if bagian:
        pattern_nama = "(?<=Bagian)(?:\s*\p{Lu}\p{L}*\s*\\n)(.*?)(?=\\n(Pasal|Paragraf))"
        pattern_isi = "Pasal.*"
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) if re.search(pattern_nama, i, re.DOTALL) else "ERROR" for i in bagian]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in bagian]
        return nama, isi
    else:
        nama = [""]
        isi = [text]
        return nama, isi

Penjelasan
1. (?<=\\n|^)(Bagian \p{Lu}.*?)(?=\\nBagian \p{Lu}|$)<br>
Digunakan untuk mengambil semua bagian<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan Bagian(spasi)[Huruf kapital] contoh (Bagian H, Bagian C, Bagian D) akan diambil<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. pengambilan karakter akan berhenti ketika bertemu \nBagian(spasi)[Huruf kapital]<br>

2. (?<=Bagian \p{Lu})(?:\p{L}*\s*\\n)(.*?)(?=\\nPasal)<br>
Digunakan untuk mengambil nama bagian<br>
Kondisi:<br>
    a. Text dimulai dengan Bagian(spasi)[Huruf kapital][Huruf atau spasi dapat diulang beberapa kali]\n tanpa diambil<br>
    b. Semua karakter selanjutnya akan diambil<br>
    c. Pengambilan karakter akan berhenti ketika bertemu \nPasal<br>

3. Pasal.*<br>
Digunakan untuk mengambil isi bagian<br>
Kondisi:<br>
    a. Text dimulai dengan Pasal dengan diambil<br>
    b. Semua karakter selanjutnya akan diambil<br>

In [901]:
for i in range(len(df_bab)):
    nama_bagian, isi_bagian = ambil_bagian(df_bab["isi_bab"][i])
    df_bagian = {"nama_bagian": nama_bagian, "isi_bagian": isi_bagian}
    display(pd.DataFrame(df_bagian))

,nama_bagian,isi_bagian
0,,Pasal 1\nDalam Undang-undang ini yang dimaksud...


,nama_bagian,isi_bagian
0,,Pasal 2\nDengan Undang-undang ini dibentuk Kab...


,nama_bagian,isi_bagian
0,,Pasal 8 \nUntuk memimpin jalannya pemerintahan...


,nama_bagian,isi_bagian
0,,Pasal 11\n(1) Pada saat terbentuknya Kabupaten...


,nama_bagian,isi_bagian
0,,Pasal 12\nPada saat terbentuknya Kabupaten Dae...


,nama_bagian,isi_bagian
0,,Pasal 17\nPada saat berlakunya Undang-undang i...


In [902]:
# nama_bagian, isi_bagian = ambil_bagian(df_bab["isi_bab"][7])
# df_bagian = {"nama_bagian": nama_bagian, "isi_bagian": isi_bagian}
# pd.DataFrame(df_bagian)

## Ambil Pasal

In [903]:
def ambil_pasal(text):
    pattern = "(?<=\\n|^)(Pasal\s*\p{N}+\s*\\n.*?)(?=\\nPasal\s*\p{N}+\s*\\n|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)(\p{N}+)"
        pattern_isi = "(?<=Pasal)(?:\s*\p{N}+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

Penjelasan
1. (?<=\\n|^)(Pasal \p{N}+\s*\\n.*?)(?=\\nPasal|$)<br>
Digunakan untuk mengambil semua pasal<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan Pasal(spasi)[Nomor satu digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n contoh (Pasal 1\n, Pasal 23     \n) akan diambil<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. Pengambilan karakter akan berhenti ketika bertemu \nPasal atau (end of string)<br>

2. (?<=Pasal)(?:\s*)(\p{N}+)<br>
Digunakan untuk mengambil no pasal<br>
Kondisi:<br>
    a. Text dimulai dengan Pasal(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan [Nomor satu digit dapat diulang beberapa kali] akan diambil<br>

3. (?<=Pasal)(?:\s*\p{N}+\s*\\n)(.*)<br>
Digunakan untuk mengambil isi pasal<br>
Kondisi:<br>
    a. Teks dimulai dengan Pasal(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan [Nomor satu digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n akan diambil (nomor pasal)<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>

In [904]:
# for i in range(len(df_bagian)):
#     no_pasal, isi_pasal = ambil_pasal(df_bagian["isi_bagian"][i])
#     df_pasal = {"no_pasal": no_pasal, "isi_pasal": isi_pasal}
#     display(pd.DataFrame(df_pasal))

In [905]:
# no_pasal, isi_pasal = ambil_pasal(df_bagian["isi_bagian"][0])
# df_pasal = {"no_pasal": no_pasal, "isi_pasal": isi_pasal}

In [906]:
# df_pasal = pd.DataFrame(df_pasal)
# df_pasal

## Ambil Ayat

In [907]:
def ambil_ayat(text):
    clean_text = re.sub('(?<=ayat)(\\n)(?=\(\p{N}\))', "", text, flags=re.DOTALL)
    ayat = re.findall('(?<=\\n|^)(\(\p{N}+\).*?)(?=\\n\(\p{N}\)|$)', clean_text, flags=re.DOTALL)
    
    if ayat:
        pattern_no = "(?<=\\n|^)(?:\()(\p{N}+)(?=\))"
        pattern_isi = "(?<=\(\p{N}+\)\s*)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in ayat]
        
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

Penjelasan
1. (?<=\\n|^)(\(\p{N}+\).*?)(?=\\n\(\p{N}\)|$)<br>
Digunakan untuk mengambil semua ayat<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai oleh (buka kurung)[nomor dapat diulang beberapa kali](tutup kurung)<br>
    c. Setelah kondisi diatas semua karakter diambil<br>
    d. Pengambilan karakter diambil hingga bertemu \n(buka kurung)[nomor dapat diulang beberapa kali](tutup kurung) atau (end of string)<br>

2. (?<=\\n|^)(?:\()(\p{N}+)(?=\))<br>
Digunakan untuk mengambil no ayat<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai oleh (buka kurung) tanpa diambil<br>
    c. Setelah kondisi diatas dimulai oleh [nomor dapat diulang beberapa kali] diambil<br>
    d. Pengambilan karakter diambil hingga bertemu (tutup kurung)<br>

3. (?<=\(\p{N}+\)\s*)(.*)<br>
Digunakan untuk mengambil isi ayat<br>
Kondisi:<br>
    a. Text dimulai dengan (buka kurung)[nomor dapat diulang beberapa kali](tutup kurung) tanpa diambil (no ayat)<br>
    b. Setelah kondisi diatas semua karakter diambil<br>

In [908]:
# for i in range(len(df_pasal)):
#     no_ayat, isi_ayat = ambil_ayat(df_pasal["isi_pasal"][i])
#     df_ayat = {"no_ayat": no_ayat, "isi_ayat": isi_ayat}
#     display(pd.DataFrame(df_ayat))

In [909]:
# no_ayat, isi_ayat = ambil_ayat(df_pasal["isi_pasal"][3])
# df_ayat = {"no_ayat": no_ayat, "isi_ayat": isi_ayat}

In [910]:
# df_ayat = pd.DataFrame(df_ayat)

# Penjelasan

In [911]:
penjelasan

'Agar setiap orang mengetahuinya, memerintahkan pengundangan\nUndang-undang ini dengan penempatannya dalam Lembaran Negara\nRepublik Indonesia.\nDisahkan di Jakarta\npada tanggal 20 April 1999\nPRESIDEN REPUBLIK INDONESIA,\nttd.\nBACHARUDDIN JUSUF HABIBIE\nDiundangkan di Jakarta\npada tanggal 20 April 1999\nMENTERI NEGARA SEKRETARIS NEGARA\nREPUBLIK INDONESIA,\nttd.\nAKBAR TANJUNG\nLEMBARAN NEGARA REPUBLIK INDONESIA TAHUN 1999 NOMOR 47\nPRESIDEN\nREPUBLIK INDONESIA\nPENJELASAN\nATAS\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 13 TAHUN 1999\nTENTANG\nPEMBENTUKAN KABUPATEN DAERAH TINGKAT II LUWU UTARA\nI. UMUM\nPropinsi daerah Tingkat I SUlawesi Selatan pada umumnya dan Kabupaten Daerah\nTingkat II Luwu pada khusunya meskipun telah menunjukkan kemajuan-kemajuan dalam\npenyelenggaraan pemerintahan, pelaksanaan pembangunan dan pelayanan kepada\nmasyarakat, namun dalam perkembangannya perlu ditingkatkan sesuai dengan potensi\ndan kebutuhan pada masa mendatang.\nPropinsi Daerah Tingkat I Sulawe

## Ambil Pasal

In [912]:
def ambil_penjelasan_pasal(text):
    pattern = "(?<=\\n|^|\s)(Pasal\s*\p{N}+.*?)(?=\\nPasal \p{N}+\s*\\n|\sPasal \p{N}+\s*\\n|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)(\p{N}+)"
        pattern_isi = "(?<=Pasal)(?:\s*\p{N}+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

In [913]:
# no_penjelasan_pasal, isi_penjelasan_pasal = ambil_penjelasan_pasal(penjelasan)
# df_penjelasan_pasal = {'no_penjelasan_pasal': no_penjelasan_pasal, 'isi_penjelasan_pasal': isi_penjelasan_pasal}

## Ambil Ayat

In [914]:
def ambil_penjelasan_ayat(text):
    pattern = "(?<=\\n|^|\s)(Ayat\s*\(\p{N}+\).*?)(?=\\nAyat\s*\(\p{N}+\)|\sAyat\s*\(\p{N}+\)|$)"
    ayat = re.findall(pattern, text, re.DOTALL)
    if ayat:
        pattern_no = "(?<=Ayat)(?:\s*\()(\p{N}+)"
        pattern_isi = "(?<=Ayat)(?:\s*\(\p{N}+\)\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in ayat]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

In [915]:
# for i in range(len(df_penjelasan_pasal)):
#     no_penjelasan_ayat, isi_penjelasan_ayat = ambil_penjelasan_ayat(df_penjelasan_pasal['isi_penjelasan_pasal'][i])
#     df_penjelasan_ayat = {'no_penjelasan_ayat': no_penjelasan_ayat, 'isi_penjelasan_ayat': isi_penjelasan_ayat}
#     display(pd.DataFrame(df_penjelasan_ayat))

In [916]:
# no_penjelasan_ayat, isi_penjelasan_ayat = ambil_penjelasan_ayat(df_penjelasan_pasal['isi_penjelasan_pasal'][61])
# df_penjelasan_ayat = {'no_penjelasan_ayat': no_penjelasan_ayat, 'isi_penjelasan_ayat': isi_penjelasan_ayat}

In [917]:
# df_penjelasan_ayat = pd.DataFrame(df_penjelasan_ayat)

In [918]:
df_badan_lst = []
lst_no_bab, lst_nama_bab, lst_isi_bab = ambil_bab(badan)

for i_bab, no_bab in enumerate(lst_no_bab):
    nama_bab = lst_nama_bab[i_bab]
    lst_nama_bagian, lst_isi_bagian = ambil_bagian(lst_isi_bab[i_bab])

    for i_bagian, nama_bagian in enumerate(lst_nama_bagian):
        lst_no_pasal, lst_isi_pasal = ambil_pasal(lst_isi_bagian[i_bagian])

        for i_pasal, no_pasal in enumerate(lst_no_pasal):
            lst_no_ayat, lst_isi_ayat = ambil_ayat(lst_isi_pasal[i_pasal])
            df_ayat = pd.DataFrame()
            df_ayat["no_ayat"] = lst_no_ayat
            df_ayat["isi_ayat"] = lst_isi_ayat
            df_ayat["no_bab"] = no_bab
            df_ayat["nama_bab"] = nama_bab
            df_ayat["nama_bagian"] = nama_bagian
            df_ayat["no_pasal"] = no_pasal
            df_badan_lst.append(df_ayat)

df_badan = pd.concat(df_badan_lst)
df_badan = df_badan[["no_bab", "nama_bab", "nama_bagian", "no_pasal", "no_ayat", "isi_ayat"]]

In [919]:
df_badan

,no_bab,nama_bab,nama_bagian,no_pasal,no_ayat,isi_ayat
0,I,KETENTUAN UMUM,,1,,Dalam Undang-undang ini yang dimaksud dengan :...
0,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,2,,Dengan Undang-undang ini dibentuk Kabupaten Da...
0,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,3,,Wilayah Kabupaten Daerah Tingkat II Luwu Utara...
0,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,4,,Dengan dibentuknya Kabupaten Daerah Tingkat II...
0,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,1,Wilayah Kabupaten Daerah Tingkat II Luwu Utar...
1,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,2,Batas wilayah sebagaimana dimaksud dalam ayat...
2,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,3,Penentuan batas wilayah Kabupaten Daerah Ting...
0,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,6,1,Dengan dibentuknya Kabupaten Daerah Tingkat I...
1,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,6,2,Penetapan Tata Ruang Wilayah Kabupaten Daerah...
0,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,7,,Ibukota Kabupaten Daerah Tingkat II Luwu Utara...


In [920]:
df_penjelasan_lst = []

lst_no_penjelasan_pasal, lst_isi_penjelasan_pasal = ambil_penjelasan_pasal(penjelasan)

for i_penjelasan_pasal, no_penjelasan_pasal in enumerate(lst_no_penjelasan_pasal):
    lst_no_penjelasan_ayat, lst_isi_penjelasan_ayat = ambil_penjelasan_ayat(lst_isi_penjelasan_pasal[i_penjelasan_pasal])
    df_penjelasan_ayat = pd.DataFrame()
    df_penjelasan_ayat['penjelasan'] = lst_isi_penjelasan_ayat
    df_penjelasan_ayat['no_ayat'] = lst_no_penjelasan_ayat
    df_penjelasan_ayat['no_pasal'] = no_penjelasan_pasal
    df_penjelasan_lst.append(df_penjelasan_ayat)

df_penjelasan = pd.concat(df_penjelasan_lst)
df_penjelasan = df_penjelasan[['no_pasal', 'no_ayat', 'penjelasan']]

In [921]:
df_penjelasan

,no_pasal,no_ayat,penjelasan
0,1,,Cukup jelas
0,2,,Wilayah Kabupaten Daerah Tingkat II Luwu Utara...
0,3,,Cukup jelas
0,4,,Cukup jelas
0,5,1,Cukup jelas
1,5,2,Peta sebagaimana dimaksud dalam ayat ini adala...
2,5,3,Penetapan batas wilayah secara pasti antara Ka...
0,6,1,Cukup jelas
1,6,2,Dalam rangka mengembangkan Kabupaten Daerah Ti...
0,7,,"Yang dimaksud dengan Masamba, sebagai Ibukota ..."


In [922]:
penjelasan

'Agar setiap orang mengetahuinya, memerintahkan pengundangan\nUndang-undang ini dengan penempatannya dalam Lembaran Negara\nRepublik Indonesia.\nDisahkan di Jakarta\npada tanggal 20 April 1999\nPRESIDEN REPUBLIK INDONESIA,\nttd.\nBACHARUDDIN JUSUF HABIBIE\nDiundangkan di Jakarta\npada tanggal 20 April 1999\nMENTERI NEGARA SEKRETARIS NEGARA\nREPUBLIK INDONESIA,\nttd.\nAKBAR TANJUNG\nLEMBARAN NEGARA REPUBLIK INDONESIA TAHUN 1999 NOMOR 47\nPRESIDEN\nREPUBLIK INDONESIA\nPENJELASAN\nATAS\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 13 TAHUN 1999\nTENTANG\nPEMBENTUKAN KABUPATEN DAERAH TINGKAT II LUWU UTARA\nI. UMUM\nPropinsi daerah Tingkat I SUlawesi Selatan pada umumnya dan Kabupaten Daerah\nTingkat II Luwu pada khusunya meskipun telah menunjukkan kemajuan-kemajuan dalam\npenyelenggaraan pemerintahan, pelaksanaan pembangunan dan pelayanan kepada\nmasyarakat, namun dalam perkembangannya perlu ditingkatkan sesuai dengan potensi\ndan kebutuhan pada masa mendatang.\nPropinsi Daerah Tingkat I Sulawe

In [923]:
df_penjelasan['pasal_ayat'] = df_penjelasan['no_pasal'] + "-" + df_penjelasan['no_ayat']
df_badan['pasal_ayat'] = df_badan['no_pasal'] + "-" + df_badan['no_ayat']

In [924]:
df_left = df_badan.merge(df_penjelasan, how="left", on="pasal_ayat", suffixes=('', '_y'))
df_left = df_left.drop(['pasal_ayat', 'no_pasal_y', 'no_ayat_y'], axis=1)

In [925]:
df_left

,no_bab,nama_bab,nama_bagian,no_pasal,no_ayat,isi_ayat,penjelasan
0,I,KETENTUAN UMUM,,1,,Dalam Undang-undang ini yang dimaksud dengan :...,Cukup jelas
1,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,2,,Dengan Undang-undang ini dibentuk Kabupaten Da...,Wilayah Kabupaten Daerah Tingkat II Luwu Utara...
2,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,3,,Wilayah Kabupaten Daerah Tingkat II Luwu Utara...,Cukup jelas
3,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,4,,Dengan dibentuknya Kabupaten Daerah Tingkat II...,Cukup jelas
4,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,1,Wilayah Kabupaten Daerah Tingkat II Luwu Utar...,Cukup jelas
5,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,2,Batas wilayah sebagaimana dimaksud dalam ayat...,Peta sebagaimana dimaksud dalam ayat ini adala...
6,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,3,Penentuan batas wilayah Kabupaten Daerah Ting...,Penetapan batas wilayah secara pasti antara Ka...
7,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,6,1,Dengan dibentuknya Kabupaten Daerah Tingkat I...,Cukup jelas
8,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,6,2,Penetapan Tata Ruang Wilayah Kabupaten Daerah...,Dalam rangka mengembangkan Kabupaten Daerah Ti...
9,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,7,,Ibukota Kabupaten Daerah Tingkat II Luwu Utara...,"Yang dimaksud dengan Masamba, sebagai Ibukota ..."


In [926]:
df_right = df_badan.merge(df_penjelasan, how="right", on="pasal_ayat", suffixes=('', '_y'))
df_right = df_right.drop(['pasal_ayat', 'no_pasal_y', 'no_ayat_y'], axis=1)

In [927]:
df_right

,no_bab,nama_bab,nama_bagian,no_pasal,no_ayat,isi_ayat,penjelasan
0,I,KETENTUAN UMUM,,1,,Dalam Undang-undang ini yang dimaksud dengan :...,Cukup jelas
1,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,2,,Dengan Undang-undang ini dibentuk Kabupaten Da...,Wilayah Kabupaten Daerah Tingkat II Luwu Utara...
2,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,3,,Wilayah Kabupaten Daerah Tingkat II Luwu Utara...,Cukup jelas
3,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,4,,Dengan dibentuknya Kabupaten Daerah Tingkat II...,Cukup jelas
4,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,1,Wilayah Kabupaten Daerah Tingkat II Luwu Utar...,Cukup jelas
5,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,2,Batas wilayah sebagaimana dimaksud dalam ayat...,Peta sebagaimana dimaksud dalam ayat ini adala...
6,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,3,Penentuan batas wilayah Kabupaten Daerah Ting...,Penetapan batas wilayah secara pasti antara Ka...
7,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,6,1,Dengan dibentuknya Kabupaten Daerah Tingkat I...,Cukup jelas
8,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,6,2,Penetapan Tata Ruang Wilayah Kabupaten Daerah...,Dalam rangka mengembangkan Kabupaten Daerah Ti...
9,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,7,,Ibukota Kabupaten Daerah Tingkat II Luwu Utara...,"Yang dimaksud dengan Masamba, sebagai Ibukota ..."


In [928]:
df = df_badan.merge(df_penjelasan, how="inner", on="pasal_ayat", suffixes=('', '_y'))
df = df.drop(['pasal_ayat', 'no_pasal_y', 'no_ayat_y'], axis=1)

In [929]:
df

,no_bab,nama_bab,nama_bagian,no_pasal,no_ayat,isi_ayat,penjelasan
0,I,KETENTUAN UMUM,,1,,Dalam Undang-undang ini yang dimaksud dengan :...,Cukup jelas
1,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,2,,Dengan Undang-undang ini dibentuk Kabupaten Da...,Wilayah Kabupaten Daerah Tingkat II Luwu Utara...
2,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,3,,Wilayah Kabupaten Daerah Tingkat II Luwu Utara...,Cukup jelas
3,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,4,,Dengan dibentuknya Kabupaten Daerah Tingkat II...,Cukup jelas
4,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,1,Wilayah Kabupaten Daerah Tingkat II Luwu Utar...,Cukup jelas
5,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,2,Batas wilayah sebagaimana dimaksud dalam ayat...,Peta sebagaimana dimaksud dalam ayat ini adala...
6,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,5,3,Penentuan batas wilayah Kabupaten Daerah Ting...,Penetapan batas wilayah secara pasti antara Ka...
7,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,6,1,Dengan dibentuknya Kabupaten Daerah Tingkat I...,Cukup jelas
8,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,6,2,Penetapan Tata Ruang Wilayah Kabupaten Daerah...,Dalam rangka mengembangkan Kabupaten Daerah Ti...
9,II,"PEMBENTUKAN, BATAS WILAYAH, DAN IBUKOTA",,7,,Ibukota Kabupaten Daerah Tingkat II Luwu Utara...,"Yang dimaksud dengan Masamba, sebagai Ibukota ..."


## UU YANG SUDAH BISA
UU1-1999<br>
UU8-1999<br>
UU9-1999<br>
UU10-1999<br>
UU11-1999<br>
UU-13-1999<br>

## UU YANG BELOM BISA
UU6-1999 (Pasalnya pake romawi)<br>
UU7-1999 (Gaada penjelasan)<br>
UU12-1999 (Penjelasan gaada ayat)<br>